In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mydata/inaturalist_12K/val/Reptilia/8f68c5f58315870b69f35bccbba75a37.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/d1dea41a26a300cad47a745d113c6af8.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/a496cceb6fe3784983a8d8e59c30f1e6.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/402bb3c00f9f397ee9f83ed9191e0d71.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/66f921bfe3df519cd3a6b6e3923ca647.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/458ca16be6a45fec3eb04d94a551cb7e.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/ee6c9d203a79b6009fc9f3abb8bb01e4.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/a78fe1dfd176763846a6132ba2ccb947.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/712ceb021f7c5df799dd81a8d86dc191.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/c136b37eed62112c4cdbb5df50dc8e3d.jpg
/kaggle/input/mydata/inaturalist_12K/val/Reptilia/e943e2e4a14e723ae5039487b097aca1.jpg
/kaggle/input/mydata/inaturalist_12K/val/Re

# Step 1 : Setting up wandb

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

In [4]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ma23m020 (snehalma23m020-iit-madras). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Step 2: Loading the inaturalist dataset 
# train set is divided into train and validation in the ratio 80-20.

In [5]:
train = '/kaggle/input/mydata/inaturalist_12K/train'
test = '/kaggle/input/mydata/inaturalist_12K/val'

In [6]:
classes = ['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

# Step 3 : Importing the iportant libraries

In [1]:
import torch     # core package
import torch.nn as nn    # to build neural network 
import torch.optim as optim  # contains optimisers
#import pytorch_lightning as pl # manage devices 
import torch.nn.functional as F
import wandb
import random
from collections import defaultdict
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader, random_split,Subset
from torchvision import transforms, datasets
#from pytorch_lightning.loggers import WandbLogger
#import pytorch_lightning as pl

Reference : [https://www.datacamp.com/tutorial/pytorch-lightning-tutorial](http://)

# Question 1:

# Step 4 : Set up

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    # To check if GPU is available

# Step 5 : Building a CNN network

In [8]:
# function to compute image size after convolution
def compute_img_size(img_w, filter_size, padding, stride):
    return (1 + (img_w - filter_size + (2 * padding)) / stride)

In [4]:
# Reference: https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/

class AdaptiveCNN(nn.Module):
    def __init__(self, input_size=224, input_channels=3, batch_size=64,num_filters=32, filter_org='same', kernel_size=[3]*5, act_fn='relu',num_neurons=256, batch_norm=True, dropout_rate=0.3,
                 learning_rate=1e-3, l2_reg=0.0):
        
        super(AdaptiveCNN, self).__init__()

        # hyperparameters
        self.batch_norm = batch_norm
        self.learning_rate = learning_rate
        self.l2_reg = l2_reg
        self.batch_size = batch_size

        # Set number of filters based on organization type
        if filter_org == 'same':   # keeping all the filters same
            filters = [num_filters] * 5
            
        elif filter_org == 'double': # doubling the filter size 
            filters = [num_filters, num_filters * 2, num_filters * 4, num_filters * 8, num_filters * 16]
        
        elif filter_org == 'half':  # halfing the filter size 
            filters = [num_filters, max(num_filters // 2, 1), max(num_filters // 4, 1), max(num_filters // 8, 1), max(num_filters // 16, 1)]
        
        else:
            raise ValueError("Invalid filter_org")

         # Mapping string for activation function
        act_func = {'relu': nn.ReLU(), 'tanh': nn.Tanh(), 'gelu': nn.GELU(), 'silu': nn.SiLU(), 'mish': nn.Mish()}
        activation = act_func[act_fn]

         # padding to keep output size constant (same padding)
        padding_vals = [k // 2 for k in kernel_size]

        # 5 convolutional blocks (using Sequential)
        # Every block: Conv2d → (Optional BatchNorm) → Activation → Dropout → MaxPool
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_channels, filters[0], kernel_size=kernel_size[0], padding=padding_vals[0]),
            nn.BatchNorm2d(filters[0]) if batch_norm else nn.Identity(),
            activation,
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(2),

            nn.Conv2d(filters[0], filters[1], kernel_size=kernel_size[1], padding=padding_vals[1]),
            nn.BatchNorm2d(filters[1]) if batch_norm else nn.Identity(),
            activation,
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(2),

            nn.Conv2d(filters[1], filters[2], kernel_size=kernel_size[2], padding=padding_vals[2]),
            nn.BatchNorm2d(filters[2]) if batch_norm else nn.Identity(),
            activation,
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(2),

            nn.Conv2d(filters[2], filters[3], kernel_size=kernel_size[3], padding=padding_vals[3]),
            nn.BatchNorm2d(filters[3]) if batch_norm else nn.Identity(),
            activation,
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(2),

            nn.Conv2d(filters[3], filters[4], kernel_size=kernel_size[4], padding=padding_vals[4]),
            nn.BatchNorm2d(filters[4]) if batch_norm else nn.Identity(),
            activation,
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(2),
        )


        
        size = input_size
        for i in range(5):
            size = compute_img_size(size, kernel_size[i], padding_vals[i], 1)
            size = size // 2 # for pooling

         # Defining fully connected layers
        self.flattened_size = int(size * size * filters[4])

        self.fc_layers = nn.Sequential(
            
            nn.Flatten(),           # flattening of 3D tensor to 1D
            nn.Linear(self.flattened_size, num_neurons),
            activation,              # Activation function
            nn.Dropout(p=dropout_rate),   # Dropout (as regularization)
            nn.Linear(num_neurons, 10)    # Final output layer (for 10 classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)  # Forward pass through conv blocks
        x = self.fc_layers(x)     # Forward pass through fully connected blocks
        return x


# You can change the value of which ever parameter you want  

In [ ]:
# Example instantiation:
model = AdaptiveCNN(
    input_size=224,
    input_channels=3,
    num_filters=64,
    filter_org='double',
    kernel_size=[3, 5, 3, 3, 3],
    act_fn='gelu',
    num_neurons=512
)


# Q2

# Step 6 : Splitting the training dataset into train set and validation set

In [ ]:
# Stratified split to ensure that each class is equally represented
def stratified_split(dataset, val_fraction=0.2, seed=42):
    
    random.seed(seed)   # Set seed for reproducibility
    
    # Create a dictionary mapping each class label to a list of its sample indices
    label_to_indices = defaultdict(list)

    
    for idx, (_, label) in enumerate(dataset.samples):
        label_to_indices[label].append(idx)

    train_indices, val_indices = [], []
    
    for label, indices in label_to_indices.items():
        n_val = int(len(indices) * val_fraction)
        random.shuffle(indices)
        val_indices.extend(indices[:n_val])
        train_indices.extend(indices[n_val:])
        
    return Subset(dataset, train_indices), Subset(dataset, val_indices)     # Return Subset objects to be used with DataLoader

# Step 7 : Loads the training dataset from the specified directory and return the stratified split

In [6]:

# Dataset loader with augmentation toggle
def load_datasets(data_dir, use_augmentation):

    # Base image transformations (always applied):
    base_transforms = [
        transforms.Resize((224, 224)),     # Resize images to 224x224 
        transforms.ToTensor(),     # Convert images to PyTorch tensors
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))     # Normalize images to [-1, 1] range using mean=0.5 and std=0.5 for all channels
    ]

    # Additional data augmentation techniques (optional):
    augment = [transforms.RandomHorizontalFlip(), transforms.RandomRotation(10)]

    # full transformation
    # If augmentation is enabled,apply augmentation transforms to base transforms
    transform = transforms.Compose(augment + base_transforms) if use_augmentation else transforms.Compose(base_transforms)

  # Load the entire dataset from the 'train' directory using ImageFolder structure
    full_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
    
    return stratified_split(full_dataset)

# Step 8 : Sweep Configuration

In [11]:
sweep_config = {
    "method": "bayes",
    "name": "updated_bayes_sweep",
    "metric": {
         "name": "val_acc", 
         "goal": "maximize"
    },
    "parameters": {
        "num_filters": {
            "values": [32, 64, 128] 
        },
        "filter_org": {
            "values": ["same", "double", "half"]
        },
        "act_fn": {
            "values": ["relu", "gelu", "silu", "mish", "tanh"]  
        },
        "dropout_rate": {
            "values": [0, 0.2, 0.3, 0.5] 
        },
        "batch_norm": {
            "values": [True, False]
        },
        "data_augmentation": {
            "values": [True, False]
        },
        "num_neurons": {
            "values": [64, 128, 256]  
        },
        "learning_rate": {
            "values": [0.0005, 0.001, 0.0001] 
        },
        "batch_size": {
            "values": [32, 64]
        },
        "kernel_size": {
            "values": [
                [3]*5,
                [3, 5, 5, 7, 7],
                [5]*5,
                [7]*5,
                [7, 5, 5, 3, 3]
            ]
        },
        "l2_reg": {
            "values": [0, 0.0005, 0.05]
        },
        "epochs": {
            "values": [15,17,20]
        }
    }
}


# Step 9 : 

In [ ]:
def train():
    
    # Initialize a new W&B run
    wandb.init()

    # Access sweep configuration
    config = wandb.config

   # naming the run using key hyperparameters
    wandb.run.name = f"sweep_{config.act_fn}_{config.num_filters}_{config.filter_org}_{config.dropout_rate}"
    
  # Load dataset 
    trainset, valset = load_datasets("/kaggle/input/mydata/inaturalist_12K", config.data_augmentation)

 # data loaders for training and validation
    train_loader = DataLoader(trainset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(valset, batch_size=config.batch_size, shuffle=False)

 # Initialize the AdaptiveCNN model using hyperparameters from wandb config
    model = AdaptiveCNN(
        input_size=224,
        input_channels=3,
        batch_size=config.batch_size,
        num_filters=config.num_filters,
        filter_org=config.filter_org,
        kernel_size=config.kernel_size,
        act_fn=config.act_fn,
        num_neurons=config.num_neurons,
        batch_norm=config.batch_norm,
        dropout_rate=config.dropout_rate,
        learning_rate=config.learning_rate,
        l2_reg=config.l2_reg
        
).to(device)


     # Define loss function (CrossEntropy for multi-class classification)
    criterion = nn.CrossEntropyLoss()

     # Define optimizer and include L2 regularization (weight decay)
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.l2_reg)

    # Learning rate scheduler 
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    
    # Training loop
    for epoch in range(config.epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        
        # Validation loop
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                loss = criterion(outputs, y)

                val_loss += loss.item() * x.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == y).sum().item()
                val_total += y.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        # Step the scheduler based on validation loss
        scheduler.step(val_loss)

       # Log metrics to W&B for the current epoch
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })


if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep_config, project="q2_assign2_exp7")

     # Start the sweep agent to run training with different hyperparameters
    wandb.agent(sweep_id, function=train, project="q2_assign2_exp7", count=30)


wandb: Agent Starting Run: edsabkzc with config:
wandb: 	act_fn: tanh
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 17
wandb: 	filter_org: double
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	l2_reg: 0.05
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 64
wandb: 	num_neurons: 256


epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
train_acc,▁▂▂▃▂▃▅▆▆▇▇▇▇████
train_loss,█▆▅▄▄▄▃▂▂▂▂▂▁▁▁▁▁
val_acc,▂▂▃▃▃▃█▅▅▆▁▆█▄▅▅▇
val_loss,█▅▅▄▅▆▃▃▇▂▆▃▁▃▃▄▃
epoch,17
train_acc,0.18188
train_loss,2.19977
val_acc,0.14957
val_loss,2.27431


wandb: Agent Starting Run: ezzn6ifv with config:
wandb: 	act_fn: silu
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0
wandb: 	epochs: 20
wandb: 	filter_org: double
wandb: 	kernel_size: [7, 7, 7, 7, 7]
wandb: 	l2_reg: 0.05
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: 64
wandb: 	num_neurons: 128


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▂▂▃▃▃▃▃▃▃▄▅▆▆▆▆▇▇██
train_loss,█▇▇▆▆▆▆▆▆▆▅▄▃▃▃▃▂▂▁▁
val_acc,▁▂▃▂▃▄▃▄▃▃▁▆▆▇▇▇▇███
val_loss,▆▆▅▇▅▆▅▆▆▆█▂▂▂▁▂▂▁▁▁
epoch,20
train_acc,0.53512
train_loss,1.3513
val_acc,0.4032
val_loss,1.73923


wandb: Agent Starting Run: 22yl5o0f with config:
wandb: 	act_fn: relu
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 20
wandb: 	filter_org: double
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	l2_reg: 0.0005
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 64
wandb: 	num_neurons: 256


...

wandb: Agent Starting Run: 69h0pv9s with config:
wandb: 	act_fn: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout_rate: 0.3
wandb: 	epochs: 15
wandb: 	filter_org: half
wandb: 	kernel_size: [7, 7, 7, 7, 7]
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 64
wandb: 	num_neurons: 128


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▃▄▄▅▅▆▆▆▇█▇▇██
train_loss,█▇▆▅▅▄▄▃▃▂▂▁▁▁▁
val_acc,▃▆█▆▅▅▃▁▂▃▃▃▃▃▃
val_loss,▆▂▁▃▂▂▅█▆▅▇▆███
epoch,15
train_acc,0.355
train_loss,1.80718
val_acc,0.14007
val_loss,2.28754


wandb: Agent Starting Run: jq1yvu2p with config:
wandb: 	act_fn: silu
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 15
wandb: 	filter_org: double
wandb: 	kernel_size: [7, 7, 7, 7, 7]
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 32
wandb: 	num_neurons: 64


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▂▃▃▄▄▅▅▅▆▆▆▇▇█
train_loss,█▇▇▆▆▅▅▅▄▄▃▃▂▂▁
val_acc,▁▄▄▅▆▅▆▆▆▇▇▆▆▇█
val_loss,█▇▅▇▃▅▃▂▂▁▁▂▃▃▁
epoch,15
train_acc,0.464
train_loss,1.52434
val_acc,0.34817
val_loss,2.3977


wandb: Agent Starting Run: bv439qth with config:
wandb: 	act_fn: relu
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout_rate: 0
wandb: 	epochs: 17
wandb: 	filter_org: half
wandb: 	kernel_size: [7, 7, 7, 7, 7]
wandb: 	l2_reg: 0.0005
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 32
wandb: 	num_neurons: 64
